# Taller 1 Buscador

Se procesan inicialmente todos los archivos que tienen las noticias.

In [ ]:
import time
import re
import math

from bs4 import BeautifulSoup   

start_time = time.time()

# Abrir el archivo inicial y tomar el documento html

archivo = open("archivosinicial/reut2-000.sgm", "r")
soup1  = BeautifulSoup(archivo, 'html.parser')
archivo.close()

documentos = []
documentos = soup1.find_all('reuters')

fileInitial = open("archivoprocesado/reut2-000.txt", "w")

for i in range(len(documentos)):
        try:          
            cadena = documentos[i].title.string.replace('\n',' ')+" "+documentos[i].body.string.replace('\n',' ')
            cadena = cadena.lower()
            cadena = cadena.replace(' reuter','')           
            fileInitial.write(cadena+"\n")                   
        except:
            pass   

fileInitial.close()

#leer archivo de stopwords, el cual tiene una sola linea con todas las palabras
#separa por ';', las palabras se guardan en un array

words = []
stopwords = open("archivosinicial/stopwords.txt", "r")
dato = stopwords.readline()
words = dato.split(';')
stopwords.close()


Se define la funcion en la cual se aplican expresiones regulares para limpiar los documentos recibidos.

In [ ]:
def processDocument(documentos,archivoProcesado):
    for cadena in documentos.readlines():
        try:                      
            cadena = re.sub(r'<.*>|[0-9]|[,*$]|[.*$]|[-*$]|[(.*)$]|[/*$]|["*$]|[\'][a-z|\W]|[+*$]|[:*$]'," ",cadena)        
            for j in range(len(words)):                                    
                cadena = re.sub(" "+words[j]+" "," ",cadena)
            archivoProcesado.write(cadena)                   
        except:
            pass        
    

archivoProcesado = open("archivoprocesado/fileDocumentos.txt", "w")
documentos =  open("archivosinicial/reut2-000.txt", "r")
processDocument(documentos,archivoProcesado)
archivoProcesado.close()
documentos.close()


Crear el diccionario de los documentos recibidos.
Se define un patron para hacer split en lo que no sea caracteres alfanumericos y se obtiene el diccionario de palabras y se aplica la funcion set para quitar duplicados.

In [2]:
import re
def getDictionary(fileDoc,fileDic):
    pattern = re.compile(r'\W+')

    fileWords = open("archivoprocesado/"+fileDoc, "r")
    dictionary = pattern.split(fileWords.read())
    dictionary = list(set(dictionary))
    dictionary.remove("")
    fileWords.close()

    #Guardar el diccionar en un archivo.
    fileDictionary = open("archivoprocesado/"+fileDic, "w")
    fileDictionary.write(str(dictionary).lstrip("[").rstrip("]").replace(", ",","))
    fileDictionary.close()
    
getDictionary("fileDocumentos.txt","dictionary.txt")

Funcion que crea la matrizTF y la guarda en un archivo, cuenta la cantidad de palabras en cada documento para crear el vector con los denominadores para obtener la matrizTFIDF.

In [3]:
import re

def setMatrizTF(fileMatriz,fileDic,fileDoc):
    
    dictionary  = open('archivoprocesado/'+fileDic).read()    
    dictionary  = dictionary.split(",")
    
    #Crear el Vector Space Model
    vectorSpace = open("archivoprocesado/"+fileMatriz, "w")
    fileFinally = open("archivoprocesado/"+fileDoc, "r")

    w=len(dictionary)
    h=len(fileFinally.readlines())

    n=0

    denominador=[0 for x in range(w)]

    linea=""

    fileFinally = open("archivoprocesado/"+fileDoc, "r")
    for lines in fileFinally.readlines(): 
        lineaAux=""
        for k in range(len(dictionary)):                     
            dictionary[k] = dictionary[k].lstrip("'").rstrip("'")                     
            patron = re.compile(r''+dictionary[k]+'')        
            count = len(patron.findall(lines))

            lineaAux = lineaAux+","+str(count)

            if count > 0:            
                denominador[k]=denominador[k]+1
            else:
                denominador[k]=denominador[k]+0

        lineaAux = lineaAux.lstrip(",")
        linea = linea+lineaAux +"\n"
        n=n+1    

    linea = linea.rstrip("\n")
    vectorSpace.write(linea)
    vectorSpace.close()
    fileFinally.close()  
    
    fileDen = open("archivoprocesado/denominador.txt", "w")
    fileDen.write(str(denominador).lstrip("[").rstrip("]").replace(", ",","))
    fileDen.close()
    
setMatrizTF("matriz_tf.txt","dictionary.txt","fileDocumentos.txt")  

In [10]:
# Obtener matriz TF-IDF
import math
import time

start_time = time.time()

den= open("archivoprocesado/denominador.txt", "r").read()
denominador = den.split(",")

matrizTF = open('archivoprocesado/matriz_tf.txt').read()
matrizTF = [item.split() for item in matrizTF.split('\n')]

w=len(denominador)
h=len(matrizTF)

matrizTFIDF = [[0 for x in range(w)] for y in range(h)]

#Creamos matriz TF-IDF
doc = []
for i in range(len(matrizTF)):
    fila = matrizTF[i]     
    doc  = str(fila).lstrip("['").rstrip("']").split(",")
    for j in range(len(doc)):  
        if(int(doc[j]) > 0):
            matrizTFIDF[i][j] = int(doc[j])*math.log(h/int(denominador[j]))
        else:    
            matrizTFIDF[i][j] = 0

print(matrizTFIDF)              
print(" ------ Tiempo de ejecucion %s -----------" % (time.time()-start_time))     

[[0, 0, 2.772588722239781, 0, 2.772588722239781, 0, 0, 0, 5.545177444479562, 1.3862943611198906, 0, 0, 0.6931471805599453, 1.3862943611198906, 0.6931471805599453, 0.6931471805599453, 4.1588830833596715, 2.772588722239781, 0.6931471805599453, 0, 6.931471805599453, 0, 0, 9.704060527839234, 1.3862943611198906, 0, 0, 2.772588722239781, 0.6931471805599453, 2.772588722239781, 1.3862943611198906, 0, 0, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 0, 0, 6.931471805599453, 0, 0, 0, 0, 0, 0, 0.0, 2.772588722239781, 1.3862943611198906, 1.3862943611198906, 0, 1.3862943611198906, 0, 12.476649250079015, 4.1588830833596715, 2.0794415416798357, 0, 0, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 0, 1.3862943611198906, 2.772588722239781, 1.3862943611198906, 3.4657359027997265, 0, 0, 0, 1.3862943611198906, 1.3862943611198906, 8.317766166719343, 0, 0, 0, 1.3862943611198906, 0, 1.3862943611198906, 1.3862943611198906, 0, 4.1588830833596715, 1.3862943611198906, 5.545177444479562